In [1]:
import pandas as pd
from quantrl_lab.data import (
    DataSourceRegistry,
    DataProcessor,
    IndicatorRegistry,
    AlpacaDataLoader, 
    YfinanceDataloader, 
    AlphaVantageDataLoader
)
from quantrl_lab.feature_selection.indicator_analysis import (
    IndicatorAnalysis,
    IndicatorAnalysisConfig,
    analyze_single_indicator,
    rank_indicator_performance
)

from quantrl_lab.feature_selection.vectorized_strategy_factory import VectorizedStrategyFactory
from quantrl_lab.data.indicators.technical_indicators import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

In [2]:
print(f"Available indicators:  {IndicatorRegistry.list_all()}")

Available indicators:  ['SMA', 'EMA', 'RSI', 'MACD', 'ATR', 'BB', 'STOCH', 'OBV']


In [3]:
# Example
supported = VectorizedStrategyFactory.get_supported_indicators()
requirements = VectorizedStrategyFactory.get_indicator_requirements('MACD')

print(f"Supported indicators: {supported}")
print(f"Requirements for MACD: {requirements}")

Supported indicators: ['SMA', 'EMA', 'RSI', 'MACD', 'ATR', 'BB', 'STOCH', 'OBV']
Requirements for MACD: {'required': ['fast', 'slow', 'signal'], 'optional': []}


In [4]:
data_loader = DataSourceRegistry()

In [5]:
olhcv_df = data_loader.get_historical_ohlcv_data(
    symbols="MU",
    start="2023-01-01",
    end="2025-01-01",
    timeframe="1d",
)

Fetching historical data for MU from 2023-01-01 to 2025-01-01 with timeframe 1d

In [6]:
indicator_configs = IndicatorAnalysis.create_indicator_configs(
    {
        'name': 'SMA',
        'params': [{'window': w} for w in [5, 10, 20, 50]]
    },
    {
        'name': 'EMA', 
        'params': [{'window': w} for w in [5, 10, 20, 50]]
    },
    {
        'name': 'RSI',
        'params': [{'window': w} for w in [7, 14, 21, 28]],
        'strategy_params': {'oversold': 30, 'overbought': 70}
    },
    {
        'name': 'MACD',
        'params': [
            {'fast': 12, 'slow': 26, 'signal': 9},
            {'fast': 5, 'slow': 35, 'signal': 5}
        ]
    },
    {
        'name': 'ATR',
        'params': [{'window': w} for w in [14, 20, 30]],
        'strategy_params': {'multiplier': 1.5}
    },
    {
        'name': 'BB',
        'params': [{'window': w, 'num_std': 2.0} for w in [20, 50]],  # Changed 'std_dev' to 'num_std'
    },
    {
        'name': 'STOCH',
        'params': [
            {'k_window': 14, 'd_window': 3, 'smooth_k': 1},
            {'k_window': 14, 'd_window': 3, 'smooth_k': 3}
        ],
        'strategy_params': {'oversold': 20, 'overbought': 80}
    },
    {
        'name': 'OBV',
        'params': [{}]
    }
)

In [7]:
from rich import print
print(indicator_configs)

{
    'SMA_5': {'indicator_params': {'window': 5}, 'strategy_params': {}},
    'SMA_10': {'indicator_params': {'window': 10}, 'strategy_params': {}},
    'SMA_20': {'indicator_params': {'window': 20}, 'strategy_params': {}},
    'SMA_50': {'indicator_params': {'window': 50}, 'strategy_params': {}},
    'EMA_5': {'indicator_params': {'window': 5}, 'strategy_params': {}},
    'EMA_10': {'indicator_params': {'window': 10}, 'strategy_params': {}},
    'EMA_20': {'indicator_params': {'window': 20}, 'strategy_params': {}},
    'EMA_50': {'indicator_params': {'window': 50}, 'strategy_params': {}},
    'RSI_7': {'indicator_params': {'window': 7}, 'strategy_params': {'oversold': 30, 'overbought': 70}},
    'RSI_14': {'indicator_params': {'window': 14}, 'strategy_params': {'oversold': 30, 'overbought': 70}},
    'RSI_21': {'indicator_params': {'window': 21}, 'strategy_params': {'oversold': 30, 'overbought': 70}},
    'RSI_28': {'indicator_params': {'window': 28}, 'strategy_params': {'oversold': 30, 'overbought': 70}},
    'MACD_12_26_9': {'indicator_params': {'fast': 12, 'slow': 26, 'signal': 9}, 'strategy_params': {}},
    'MACD_5_35_5': {'indicator_params': {'fast': 5, 'slow': 35, 'signal': 5}, 'strategy_params': {}},
    'ATR_14': {'indicator_params': {'window': 14}, 'strategy_params': {'multiplier': 1.5}},
    'ATR_20': {'indicator_params': {'window': 20}, 'strategy_params': {'multiplier': 1.5}},
    'ATR_30': {'indicator_params': {'window': 30}, 'strategy_params': {'multiplier': 1.5}},
    'BB_20_2.0': {'indicator_params': {'window': 20, 'num_std': 2.0}, 'strategy_params': {}},
    'BB_50_2.0': {'indicator_params': {'window': 50, 'num_std': 2.0}, 'strategy_params': {}},
    'STOCH_14_3_1': {
        'indicator_params': {'k_window': 14, 'd_window': 3, 'smooth_k': 1},
        'strategy_params': {'oversold': 20, 'overbought': 80}
    },
    'STOCH_14_3_3': {
        'indicator_params': {'k_window': 14, 'd_window': 3, 'smooth_k': 3},
        'strategy_params': {'oversold': 20, 'overbought': 80}
    },
    'OBV': {'indicator_params': {}, 'strategy_params': {}}
}

In [10]:
settings = IndicatorAnalysisConfig(
    initial_capital=100000,
    transaction_cost=0.001,
    position_sizing='full'
)

comparison_df = rank_indicator_performance(
    data=olhcv_df,
    indicators=indicator_configs,
    config=settings
)

comparison_df

/Users/hanyuwu/Study/QuantRL-Lab/src/quantrl_lab/feature_selection/vectorized_strategy_implementations.py:81: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  signals = signals.replace(SignalType.HOLD.value, pd.NA).ffill().fillna(SignalType.HOLD.value).infer_objects(copy=False)
/Users/hanyuwu/Study/QuantRL-Lab/src/quantrl_lab/feature_selection/vectorized_strategy_implementations.py:81: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  signals = signals.replace(SignalType.HOLD.value, pd.NA).ffill().fillna(SignalType.HOLD.value).infer_objects(copy=False)
/Users/h

,indicator,total_return,annual_return,volatility,sharpe_ratio,max_drawdown,win_rate,profit_factor,sortino_ratio,calmar_ratio,final_value,total_trades
20,STOCH_14_3_3,2.393563,0.846652,0.696958,1.186087,-0.365858,0.247505,1.775653,2.329653,2.314152,339356.34881,25
8,RSI_7,2.170450,0.784673,0.756726,1.010503,-0.279018,0.249501,1.756396,2.068405,2.812267,317045.01906,19
19,STOCH_14_3_1,1.572400,0.606892,0.755750,0.776569,-0.365863,0.243513,1.587365,1.621834,1.658797,257240.00070,25
3,SMA_50,1.451664,0.568579,0.890135,0.616287,-0.265825,0.313373,1.478263,2.355420,2.138918,245166.38820,30
13,MACD_5_35_5,0.438158,0.200101,0.304294,0.591864,-0.352165,0.239521,1.218901,0.595144,0.568202,143815.81023,56
21,OBV,1.182903,0.479764,0.792845,0.579892,-0.305033,0.249501,1.468598,1.483257,1.572829,218290.30119,70
11,RSI_28,0.280235,0.132032,0.309948,0.361453,-0.366186,0.197605,1.166090,0.372773,0.360559,128023.49811,1
0,SMA_5,0.282964,0.133243,0.323089,0.350500,-0.280734,0.249501,1.141707,0.455271,0.474622,128296.44513,130
4,EMA_5,0.281882,0.132762,0.323125,0.348975,-0.280141,0.265469,1.137601,0.462491,0.473913,128188.18350,130
5,EMA_10,0.225001,0.107245,0.346531,0.251767,-0.341748,0.277445,1.116896,0.304015,0.313814,122500.09728,96
